In [1]:
import os
import time
import pickle
import sqlite3
import numpy as np
import pandas as pd

In [2]:
def folderChecker(my_folder='./my_folder'):
    ## ------- simply clean up the folder path -------
    if my_folder is None:
        my_folder='./tmp'
    elif '/' not in my_folder:
        my_folder = os.path.join(os.getcwd(), my_folder)

    ## ------- Check if the folder exists -------
    check_folder = os.path.isdir(my_folder)
    # os.path.exists(dir_outputs)
    # If the folder does not exist, create it
    if not check_folder:
        os.makedirs(my_folder)
        print(f"\tCreated folder: {my_folder}")
    else:
        print(f'\t{my_folder} is existing')

    return my_folder

In [3]:
file_mmpdb = '../results/Compounds_All.mmpdb'
tmp_folder = folderChecker('./tmp')
# dataDict_csvFiles = Step_4_extract_data_from_DB(file_mmpdb, tmp_folder, save=False)

	./tmp is existing


In [4]:
dataDict_csvFiles = {'pair': './tmp/DB_tables/DB_table_pair.csv',
 'compound': './tmp/DB_tables/DB_table_compound.csv',
 'compound_property': './tmp/DB_tables/DB_table_compound_property.csv',
 'property_name': './tmp/DB_tables/DB_table_property_name.csv',
 'constant_smiles': './tmp/DB_tables/DB_table_constant_smiles.csv',
 'rule': './tmp/DB_tables/DB_table_rule.csv',
 'rule_smiles': './tmp/DB_tables/DB_table_rule_smiles.csv',
 'rule_environment': './tmp/DB_tables/DB_table_rule_environment.csv',
 'rule_environment_statistics': './tmp/DB_tables/DB_table_rule_environment_statistics.csv',
 'environment_fingerprint': './tmp/DB_tables/DB_table_environment_fingerprint.csv'}

In [5]:
db_table_name = "pair"
dataTable_pair = pd.read_csv(dataDict_csvFiles[db_table_name]).rename(columns={'id': 'pair_id'})
print(f"There are total {dataTable_pair.shape[0]} rows in this <{db_table_name}> table")
dataTable_pair.head(3)

There are total 46568964 rows in this <pair> table


,pair_id,rule_environment_id,compound1_id,compound2_id,constant_id
0,1,1,1,2,1
1,2,2,1,2,1
2,3,3,1,2,1


In [6]:
db_table_name = "compound"
dataTable_cmpd = pd.read_csv(dataDict_csvFiles[db_table_name])
dataTable_cmpd.drop(columns=['clean_smiles', 'clean_num_heavies'], inplace=True)
dataTable_cmpd.rename(columns={'id': 'cmpd_id', 'public_id': 'mol_id', 'input_smiles': 'Structure'}, inplace=True)
print(f"There are total {dataTable_cmpd.shape[0]} rows in this <{db_table_name}> table")
dataTable_cmpd.head(3)

There are total 301617 rows in this <compound> table


,cmpd_id,mol_id,Structure
0,1,KT-0261652,C/C(=C\c1ccccc1)C(=O)NCCc1ccccc1
1,2,KT-0246540,O=C(NCCc1ccccc1)/C(=C\c1ccccc1)NC(=O)c1ccc(Br)o1
2,3,KT-0295265,COc1ccc(C(=O)N/C(=C\c2ccccc2)C(=O)N2CCOCC2)cc1


In [7]:
dataTable_pair = dataTable_pair.merge(right=dataTable_cmpd, left_on='compound1_id', right_on='cmpd_id', how='left')
dataTable_pair.rename(columns={col: f'From_{col}' for col in dataTable_cmpd.columns}, inplace=True)

dataTable_pair = dataTable_pair.merge(right=dataTable_cmpd, left_on='compound2_id', right_on='cmpd_id', how='left')
dataTable_pair.rename(columns={col: f'To_{col}' for col in dataTable_cmpd.columns}, inplace=True)

dataTable_pair.drop(columns=['From_cmpd_id', 'To_cmpd_id'], inplace=True)
dataTable_pair.head(3)

,pair_id,rule_environment_id,compound1_id,compound2_id,constant_id,From_mol_id,From_Structure,To_mol_id,To_Structure
0,1,1,1,2,1,KT-0261652,C/C(=C\c1ccccc1)C(=O)NCCc1ccccc1,KT-0246540,O=C(NCCc1ccccc1)/C(=C\c1ccccc1)NC(=O)c1ccc(Br)o1
1,2,2,1,2,1,KT-0261652,C/C(=C\c1ccccc1)C(=O)NCCc1ccccc1,KT-0246540,O=C(NCCc1ccccc1)/C(=C\c1ccccc1)NC(=O)c1ccc(Br)o1
2,3,3,1,2,1,KT-0261652,C/C(=C\c1ccccc1)C(=O)NCCc1ccccc1,KT-0246540,O=C(NCCc1ccccc1)/C(=C\c1ccccc1)NC(=O)c1ccc(Br)o1


In [8]:
db_table_name = "property_name"
dataTable_propName = pd.read_csv(dataDict_csvFiles[db_table_name]).rename(columns={'id': 'propName_id'})
print(f"There are total {dataTable_propName.shape[0]} rows in this <{db_table_name}> table")
dataDict_propName = dict(zip(dataTable_propName['propName_id'], dataTable_propName['name']))
print(dataDict_propName)

db_table_name = "compound_property"
dataTable_prop = pd.read_csv(dataDict_csvFiles[db_table_name])
dataTable_prop.rename(columns={'id': 'prop_id'}, inplace=True)
print(f"There are total {dataTable_prop.shape[0]} rows in this <{db_table_name}> table")
# dataTable_prop.head(3)

# Pivot the DataFrame
dataTable_prop_pivot = dataTable_prop.pivot(index='compound_id', columns='property_name_id', values='value')
dataTable_prop_pivot.rename(columns=dataDict_propName, inplace=True)
dataTable_prop_pivot.reset_index(names='cmpd_id', inplace=True)
dataTable_prop_pivot.head(3)

There are total 11 rows in this <property_name> table
{0: 'Permeability', 1: 'Efflux', 2: 'Bioavailability', 3: 'Cl_obs', 4: 'hERG_IC50', 5: 'hERG_eIC50', 6: 'hERG_mixedIC50', 7: 'estFa', 8: 'MW', 9: 'bpKa1', 10: 'logD'}
There are total 920744 rows in this <compound_property> table


property_name_id,cmpd_id,Permeability,Efflux,Bioavailability,Cl_obs,hERG_IC50,hERG_eIC50,hERG_mixedIC50,estFa,MW,bpKa1,logD
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,265.356,1.2654,3.8340
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,439.309,0.9486,3.6065
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,366.417,0.9882,2.5080


In [9]:
dataTable_pair = dataTable_pair.merge(right=dataTable_prop_pivot, left_on='compound1_id', right_on='cmpd_id', how='left')
dataTable_pair.rename(columns={col: f'From_{col}' for col in dataTable_prop_pivot.columns}, inplace=True)

dataTable_pair = dataTable_pair.merge(right=dataTable_prop_pivot, left_on='compound2_id', right_on='cmpd_id', how='left')
dataTable_pair.rename(columns={col: f'To_{col}' for col in dataTable_prop_pivot.columns}, inplace=True)

dataTable_pair.drop(columns=['From_cmpd_id', 'To_cmpd_id'], inplace=True)

dataTable_pair.head(3)

,pair_id,rule_environment_id,compound1_id,compound2_id,constant_id,From_mol_id,From_Structure,To_mol_id,To_Structure,From_Permeability,...,To_Efflux,To_Bioavailability,To_Cl_obs,To_hERG_IC50,To_hERG_eIC50,To_hERG_mixedIC50,To_estFa,To_MW,To_bpKa1,To_logD
0,1,1,1,2,1,KT-0261652,C/C(=C\c1ccccc1)C(=O)NCCc1ccccc1,KT-0246540,O=C(NCCc1ccccc1)/C(=C\c1ccccc1)NC(=O)c1ccc(Br)o1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,439.309,0.9486,3.6065
1,2,2,1,2,1,KT-0261652,C/C(=C\c1ccccc1)C(=O)NCCc1ccccc1,KT-0246540,O=C(NCCc1ccccc1)/C(=C\c1ccccc1)NC(=O)c1ccc(Br)o1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,439.309,0.9486,3.6065
2,3,3,1,2,1,KT-0261652,C/C(=C\c1ccccc1)C(=O)NCCc1ccccc1,KT-0246540,O=C(NCCc1ccccc1)/C(=C\c1ccccc1)NC(=O)c1ccc(Br)o1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,439.309,0.9486,3.6065


In [ ]:
dataTable_pair = dataDict_tables["pair"]
dbTable_cmpd = dataDict_tables["compound"]
dbTable_propName = dataDict_tables["property_name"]
dbTable_propValue = dataDict_tables["compound_property"]

In [4]:
def Step_5_MMPs_DataClean(dataDict_tables, add_symetric=True):
    ## count time
    beginTime = time.time()
    print(f"5. Now clean up the MMPs data ...")
    ## ------------------------------------------------------------------

    ## get the individual database Tables
    dataTable_pair = dataDict_tables["pair"]
    dbTable_cmpd = dataDict_tables["compound"]
    dbTable_propName = dataDict_tables["property_name"]
    dbTable_propValue = dataDict_tables["compound_property"]

    ## ------------- build the dataDict of pairs -------------
    print(f"\tNow start cleanning up the dataDict of pairs ...\n")
    dataDict = {}
    for idx in dataTable_pair.index:
        pair_idx = dataTable_pair['id'][idx]
        cid_1 = int(dataTable_pair['compound1_id'][idx])
        cid_2 = int(dataTable_pair['compound2_id'][idx])
        const_id = dataTable_pair['constant_id'][idx]
        rule_env_id = dataTable_pair['rule_environment_id'][idx]

        ## initialize the sub-dict
        pair_info = f"{cid_1}==>{cid_2}"
        try:
            pair_list = sorted([cid_1, cid_2], reverse=False)
        except Exception as e:
            pass    
        
        if pair_info not in dataDict:
            ## add pair basic info
            dataDict[pair_info] = {}
            dataDict[pair_info]["pair_info"] = pair_info
            dataDict[pair_info]["pair_id"] = f"({min([cid_1, cid_2])},{max([cid_1, cid_2])})"
            dataDict[pair_info]["compound1_id"] = cid_1
            dataDict[pair_info]["compound2_id"] = cid_2
            dataDict[pair_info]["pair_detail"] = {}

            ## add compound info
            dataDict[pair_info]["From_mol_id"] = dbTable_cmpd['public_id'][cid_1]
            dataDict[pair_info]["To_mol_id"] = dbTable_cmpd['public_id'][cid_2]
            smi_1, smi_2 = dbTable_cmpd['input_smiles'][cid_1], dbTable_cmpd['input_smiles'][cid_2]
            dataDict[pair_info]["From_Structure"] = smi_1
            dataDict[pair_info]["To_Structure"] = smi_2
            
            ## add shared structure
            # dataDict[pair_info]["SharedSubstructure"] = fun_tbd(smi_1, smi_2)

            ## add compound prop info
            for prop_id in dbTable_propName.index:
                prop_name = dbTable_propName['name'][prop_id]
                dataDict[pair_info][f"From_{prop_name}"] = _findPropValue(dbTable_propValue, cid_1, prop_id, average=True)
                dataDict[pair_info][f"To_{prop_name}"] = _findPropValue(dbTable_propValue, cid_2, prop_id, average=True)
                ## add delta value change
                try:
                    delta_value = dataDict[pair_info][f"To_{prop_name}"] - dataDict[pair_info][f"From_{prop_name}"]
                except Exception as e:
                    delta_value = np.nan
                    
                dataDict[pair_info][f"Delta_{prop_name}"] = delta_value

        ## add pair details information (constant part)
        if const_id not in dataDict[pair_info]["pair_detail"]:
            dataDict[pair_info]["pair_detail"][const_id] = []
        
        ## add pair details information (rule_env)
        if rule_env_id not in dataDict[pair_info]["pair_detail"][const_id]:
            dataDict[pair_info]["pair_detail"][const_id].append(rule_env_id)
    print(f"\t\tOriginal num_pairs in dataDict: {len(dataDict)}\n")
    
    ## ------------- add the symetric pairs if not exist -------------
    tran_smi = True
    radius = 0    # [0, 1, 2, 3, 4, 5]
    sele_rule = "max"    # ["max", "min"]
    if tran_smi:
        for pair_info in dataDict:
            pair_detail = dataDict[pair_info]["pair_detail"]
            const_smi_sele, from_smiles, to_smiles = _findTranSmi(pair_detail, dataDict_tables, sele_rule=sele_rule, radius=radius)
            dataDict[pair_info]["constant_smiles"] = const_smi_sele
            dataDict[pair_info]["from_smiles"] = from_smiles
            dataDict[pair_info]["to_smiles"] = to_smiles
        
    ## ------------- add the symetric pairs if not exist -------------
    if add_symetric:
        print(f"\t\tNow adding symetric pairs ...")
        list_pair_info_4loop = copy.deepcopy(list(dataDict.keys()))
        list_pair_info_4check = copy.deepcopy(list(dataDict.keys()))
        for pair_info in list_pair_info_4loop:
            if pair_info in list_pair_info_4check:
                list_pair_info_4check.remove(pair_info)

                ## reverse pair
                cid_1, cid_2 = pair_info.split("==>")
                pair_info_revs = f"{cid_2}==>{cid_1}"
                if pair_info_revs in list_pair_info_4check:
                    list_pair_info_4check.remove(pair_info_revs)
                else:
                    ## if reversed pair not in check list, add it in the dict
                    dataDict[pair_info_revs] = {}
                    dataDict[pair_info_revs]["pair_info"] = pair_info_revs
                    dataDict[pair_info_revs]["pair_id"] = dataDict[pair_info]["pair_id"]
                    dataDict[pair_info_revs]["constant_smiles"] = dataDict[pair_info]["constant_smiles"]
                    dataDict[pair_info_revs]["pair_detail"] = {key: [] for key in dataDict[pair_info]["pair_detail"]}

                    dataDict[pair_info_revs]["From_mol_id"] = dataDict[pair_info]["To_mol_id"]
                    dataDict[pair_info_revs]["To_mol_id"] = dataDict[pair_info]["From_mol_id"]
                    dataDict[pair_info_revs]["From_Structure"] = dataDict[pair_info]["To_Structure"]
                    dataDict[pair_info_revs]["from_smiles"] = dataDict[pair_info]["to_smiles"]
                    dataDict[pair_info_revs]["to_smiles"] = dataDict[pair_info]["from_smiles"]
                    
                    for tmp_key in dataDict[pair_info]:
                        if tmp_key[0:6] == 'Delta_':
                            try:
                                delta_symetric = dataDict[pair_info][tmp_key] * -1
                            except Exception as e:
                                delta_symetric = np.nan
                            dataDict[pair_info_revs][tmp_key] = delta_symetric

                        elif tmp_key[0:5] == 'From_':
                            tmp_key_reverse = 'To_' + tmp_key[5:]
                            dataDict[pair_info_revs][tmp_key_reverse] = dataDict[pair_info][tmp_key]
                        
                        elif tmp_key[0:3] == "To_":
                            tmp_key_reverse = 'From_' + tmp_key[3:]
                            dataDict[pair_info_revs][tmp_key_reverse] = dataDict[pair_info][tmp_key]
                        else:
                            pass
            else:
                ## this pair was removed from check list because it's the revs pair of another pair
                pass
        print(f"\t\tNew num_pairs in symetric dataDict: {len(dataDict)}")
    
    ## ------------------------------------------------------------------
    costTime = time.time()-beginTime
    print(f"==> Step 5 <Final data clean> complete, costs time = %ds ................\n" % (costTime))
    return dataDict

In [ ]:
dataDict = Step_5_MMPs_DataClean(dataDict_tables, add_symetric=True)